In [ ]:
!pip install wptools
!pip install fa2
!pip install tweepy

     |████████████████████████████████| 225kB 16.4MB/s 
  Created wheel for pycurl: filename=pycurl-7.43.0.6-cp36-cp36m-linux_x86_64.whl size=284336 sha256=98011af50dcdf8aba680d7472dd381d4213fa66d8adcdc8bc8587de594e1c87c
  Stored in directory: /root/.cache/pip/wheels/d9/a1/7b/2894883fd5c69a3ba684a49d255466d1e924b3cf58cedc1ade
Successfully built pycurl


In [ ]:
from bs4 import BeautifulSoup
import datetime
from fa2 import ForceAtlas2
import json
import matplotlib.pyplot as plt
import networkx as nx
import nltk
from nltk.probability import FreqDist
import numpy as np
import pandas as pd
from PIL import Image
import re
import time
import tweepy
from urllib.parse import quote
import urllib3
import wptools
import wordcloud
from wordcloud import ImageColorGenerator, WordCloud


urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# Get the most followed twitter profiles by scraping trackanalytics.com
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = 'https://www.trackalytics.com/the-most-followed-twitter-profiles/page/'

user_list = []

for num in range(1, 655)[100:150]:
    try:
        query = url + str(num) + '/'
        response = urllib3.PoolManager().request('GET', query).data

        # Avoid decoding warnings
        FromRaw = lambda r: r if isinstance(r, str) else r.decode('utf-8', 'ignore')

        soup = BeautifulSoup(FromRaw(response), 'html.parser')
        rows = soup.find_all('tr')[1:]
        for row in rows:
            try:
                new = BeautifulSoup(str(row), 'html.parser').find_all('a')[1]
                screen_name = new.get('title')#.title() # normalise capitalization
                username = "@" + new.get('href').split('/')[-2] # get username from link

                user_list.append([screen_name, username])

            except Exception as e:
                print('link error:', i, e)
                continue
    except Exception as e:
        print('page error:', num, e)
        continue

In [ ]:
def search_wiki(search_word):
    # Compile search query
    query = 'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=' + quote(search_word.replace(' ', '_'))
    try:
        # Query wikipedia
        wikiresponse = urllib3.PoolManager().request('GET', query)
        wikijson2 = json.loads(wikiresponse.data)

        results_list = []

        for i in wikijson2['query']['search']:
            try:
                results_list.append(i['title'])
            except:
                print('no results:', i)
                continue
        return results_list
    except Exception as e:
        print('Query error:', search_word, e)
        return []

In [ ]:
def query_birthday(wiki_link):
    page = wptools.page(wiki_link, verbose=False, silent=True)
    page.get_parse()
    print()
    birthday = page.data['infobox']['birth_date']
    print(birthday)
    birthday = re.search('\d{4}\|\d+\|\d+', birthday).group()
    date = datetime.datetime.strptime(birthday, "%Y|%m|%d").date()
    return date

In [ ]:
def zodiac_sign(date): 
    # Source: https://www.geeksforgeeks.org/program-display-astrological-sign-zodiac-sign-given-date-birth/
    # checks month and date within the valid range of a specified zodiac 
    # Parameters: date
    # Returns the zodiac sign based on the date of birth
    
    month = date.month
    day = date.day

    if month == 12: 
        astro_sign = 'Sagittarius' if (day < 22) else 'Capricorn'
        
    elif month == 1: 
        astro_sign = 'Capricorn' if (day < 20) else 'Aquarius'
        
    elif month == 2: 
        astro_sign = 'Aquarius' if (day < 19) else 'Pisces'
        
    elif month == 3: 
        astro_sign = 'Pisces' if (day < 21) else 'Aries'
        
    elif month == 4: 
        astro_sign = 'Aries' if (day < 20) else 'Taurus'
        
    elif month == 5: 
        astro_sign = 'Taurus' if (day < 21) else 'Gemini'
        
    elif month == 6: 
        astro_sign = 'Gemini' if (day < 21) else 'Cancer'
        
    elif month == 7: 
        astro_sign = 'Cancer' if (day < 23) else 'Leo'
        
    elif month == 8: 
        astro_sign = 'Leo' if (day < 23) else 'Virgo'
        
    elif month == 9: 
        astro_sign = 'Virgo' if (day < 23) else 'Libra'
        
    elif month == 10: 
        astro_sign = 'Libra' if (day < 23) else 'Scorpio'
        
    elif month == 11: 
        astro_sign = 'Scorpio' if (day < 22) else 'Sagittarius'
        
    return astro_sign 


def zodiac_element(zodiac_sign):
    # Parameters: zodiac_sign
    # Returns the element of the sign

    earth = ['Capricorn', 'Taurus', 'Virgo' ]
    water = ['Cancer', 'Pisces', 'Scorpio' ]
    fire = ['Leo', 'Aries', 'Sagittarius']
    air = ['Libra', 'Gemini', 'Aquarius']

    if zodiac_sign in earth:
        return 'Earth'
    elif zodiac_sign in water:
        return 'Water'
    elif zodiac_sign in fire:
        return 'Fire'
    elif zodiac_sign in air:
        return 'Air'
    else:
        return ''

In [ ]:
info = {}
for user in user_list:
    # Get name and twitter username from user
    name, username = user[:2]

    # Find Wiki pages corresponding to twitter page
    pages = []
    pages += (search_wiki(username))
    pages += (search_wiki(name))
    pages = [p for p in pages if 'List of' not in p]
    
    ### Select wiki_link from pages ###
    wiki_link = ''

    # If twitter name (first and last) appears in searches, these are prioritized
    first_name = name.split(' ')[0]
    last_name = name.split(' ')[-1]
    name_matches = [p for p in pages if first_name.lower() in p.lower() and last_name.lower() in p.lower() and '&' not in p]
    username_matches = [p for p in pages if (p.split(' ')[0].lower() in username and p.split(' (')[0].split(' ')[-1].lower() in username) or (p.split(' ')[0].lower() in name and p.split(' (')[0].split(' ')[-1].lower() in name)]

    # Sort by length as celebrities often have several related pages
    matches = sorted(name_matches, key=len)

    if len(matches) > 0:
        wiki_link = matches[0] # best name match
    elif len(pages) > 0: 
        wiki_link = pages[0] # first result from query
    else:
        # No pages were found
        continue

    # Extract birthday from wiki_link
    try:
        birthday = query_birthday(wiki_link)
    except Exception as e:
        print(user)
        print(wiki_link)
        try:
            # second and last attempt, this time prioritizing
            new_matches = [m for m in username_matches+name_matches+pages if not m is wiki_link]
            wiki_link = new_matches[0] # best name match
            print(wiki_link)
            birthday = query_birthday(wiki_link)
        except:
            print('failed')
            # print('bday error:', wiki_link, user, e)
            # print('FAILURE:' wiki_link, user)
            # No birthday found
            continue

    # Calculate zodiac from birthday
    zodiac = zodiac_sign(birthday)
    element = zodiac_element(zodiac)
    # print('SUCCESS:', wiki_link, user)
    info[name] = [name, username, 0, birthday, zodiac, element, wiki_link] 



['El Gordo y La Flaca', '@elgordoylaflaca']
El Gordo y la Flaca
La Flaca

failed

{{birth date and age|df|=|yes|1978|05|22}}

['Mariano Iudica', '@iudica']
Phylum
Generation X

failed

{{Birth date and age|1969|8|18}}

['MAJA', '@dprincessmaja']
Maja
Maja (genus)

failed

['ToysRUs', '@toysrus']
Toys "R" Us
Toys "R" Us

failed

['ABC.es', '@abc_es']
ABC (newspaper)
ABC (newspaper)

failed

['NCAA', '@ncaa']
NCAA Division I
College football national championships in NCAA Division I FBS

failed

{{birth date and age|1983|2|20}}

{{birth date and age|1992|10|27|df|=|yes}}

{{Birth date and age|1962|11|23|df|=|y}}

{{birth date and age|mf|=|yes|1982|3|8}}

['Tobi', '@tobjizzle']
Tobi
Tobi Adegboyega

{{Birth year and age|1980}}
failed

{{Birth date and age|1966|04|13}}

{{birth date and age|1945|12|01}}

{{birth date and age|1981|7|24|df|=|y}}

{{birth date and age|1980|07|11|df|=|yes}}

{{birth date and age|1954|09|30}}

{{Birth date and age|1990|06|06|mf|=|yes}}

['indy barends', '@indy

In [ ]:
columns = ['Name', 'Handle', 'FollowersCount', 'Birthday', 'ZodiacSign', 'Element', 'WikiLink']
df = pd.DataFrame(list(info.values()), columns=columns)

print(df.head())

            Name         Handle  ...  Element             WikiLink
0   Pawan Kalyan   @pawankalyan  ...    Earth         Pawan Kalyan
1   stc السعودية       @stc_ksa  ...    Earth  Mohammad Al-Sahlawi
2         Jr NTR     @tarak9999  ...    Earth   N. T. Rama Rao Jr.
3  Kevin Aprilio  @apriliokevin  ...      Air       Raisa (singer)
4         Fergie        @fergie  ...     Fire      Fergie (singer)

[5 rows x 7 columns]


In [ ]:
print(len(user_list))
print(len(info.values()))

1250
767


Maybe manually add links to some celebs that are hard to find?? and how should links be selected?


In [ ]:
credentials = {}
credentials['CONSUMER_KEY'] = 'Y1CqyEbJGKrqdjViOPggeDnAW'
credentials['CONSUMER_SECRET'] = 'md3hYCmmBVyvBbqRcLyyt4dnV1QFwbjElO8dJyXnpyqDS3whJ4'
credentials['ACCESS_TOKEN'] = '705776698785189888-EuyFzSSYjUmEFsVVcVFMR8ZmqMvS6zv'
credentials['ACCESS_SECRET'] = 'NhLEmGQbziKOIUKieckCvyDcWG5Uu9s92HfNlMjJhqhG6'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [ ]:
# Stetting twitter API
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

api = tweepy.API(auth)#, wait_on_rate_limit=True)

In [ ]:
# Create the nodes
G = nx.DiGraph()
G.add_nodes_from(df.Handle)

zodiac_dict = {}
element_dict = {}

df.drop_duplicates(subset ="Handle", inplace = True) 

for name in df.Handle:
    zodiac_dict[name] = df.loc[df['Handle'] == name].ZodiacSign.values[0]
    element_dict[name] = df.loc[df['Handle'] == name].Element.values[0]


nx.set_node_attributes(G, zodiac_dict, "zodiac")
nx.set_node_attributes(G, element_dict, "element")

In [ ]:
# TODO: delete

# Query follows from twitter API
for twitter_user in df.Handle[:5]:
    following = []

    # Getting their following list and creating the links in the network    
    for user in tweepy.Cursor(api.friends, screen_name=twitter_user).items(3):
        if user.screen_name in G.nodes() and twitter_user in G.nodes():
            G.add_edge(twitter_user, user.screen_name)
            # Add followers to df.Following
            following.append(user.screen_name)
            print(twitter_user, user.screen_name)
    print(twitter_user, following)
    df.loc[df['Handle'] == twitter_user].append(following)
df.head

@pawankalyan []
@stc_ksa []
@tarak9999 []
@apriliokevin []
@fergie []


<bound method NDFrame.head of                    Name          Handle  ...  Element             WikiLink
0          Pawan Kalyan    @pawankalyan  ...    Earth         Pawan Kalyan
1          stc السعودية        @stc_ksa  ...    Earth  Mohammad Al-Sahlawi
2                Jr NTR      @tarak9999  ...    Earth   N. T. Rama Rao Jr.
3         Kevin Aprilio   @apriliokevin  ...      Air       Raisa (singer)
4                Fergie         @fergie  ...     Fire      Fergie (singer)
..                  ...             ...  ...      ...                  ...
762  chaitanya akkineni  @chay_akkineni  ...     Fire       Naga Chaitanya
763        bubba watson    @bubbawatson  ...    Water         Bubba Watson
764        Awal Ashaari    @awalashaari  ...     Fire         Awal Ashaari
765             ASHANTI        @ashanti  ...      Air     Ashanti (singer)
766           Jay Bilas       @jaybilas  ...    Earth            Jay Bilas

[767 rows x 7 columns]>

In [ ]:
### TWITTER FOLLOWING TASKS - TODO: Solve problems related with request limit

# Management for the request limit
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

following = {}

# Query following from twitter API
for twitter_user in df.Handle[:5]:
    following[twitter_user] = []
    
    # Getting their following list and creating the links in the network    
    for user in limit_handled(tweepy.Cursor(api.friends, screen_name=twitter_user).items()):
        if user.screen_name in G.nodes() and twitter_user in G.nodes():
            G.add_edge(twitter_user, user.screen_name)
            following[twitter_user].append(user.screen_name.text)

# Add following to df
df['Following'] = df['Handle'].map(following)  
print(df.head) 

TweepError: ignored

In [ ]:
df.head
df.to_csv("../100p_df.csv")


In [ ]:
# Remove isolates
isolated = list(nx.isolates(G))
G.remove_nodes_from(isolated)

# Select giant connected component
largest_cc = max(nx.weakly_connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

# Create undirected graph
U = G.to_undirected()

ValueError: ignored

In [ ]:
print("The number of nodes in the network is: ", G.number_of_nodes())
print("The number of links in the network is: ", G.number_of_edges())

The number of nodes in the network is:  0
The number of links in the network is:  0


In [ ]:
in_degree = [value for key, value in list(G.in_degree())]
plt.hist(in_degree, bins=range(np.min(in_degree), np.max(in_degree)+1), alpha=0.5, label="in")

out_degree = [value for key, value in list(G.out_degree())]
plt.hist(out_degree, bins=range(np.min(out_degree), np.max(out_degree)+1), alpha=0.5, label="out")

plt.title("title")
plt.ylabel("Frequency")
plt.xlabel("Degree")
plt.legend()
plt.figtext(
    0.5, 
    -0.05, 
    'hi', 
    wrap=True, 
    horizontalalignment='center', 
    fontsize=12
)
plt.show()


ValueError: ignored

In [ ]:
# create empty list for node colors
node_color = []

# for each node in the graph
for node in G.nodes(data=True):
    # if the node has the attribute group1
    if node[1]['zodiac'] is 'Aries':
        node_color.append('#FFFF66')
    elif node[1]['zodiac'] is 'Taurus':
        node_color.append('gold')
    elif node[1]['zodiac'] is 'Gemini':
        node_color.append('orange')
    elif node[1]['zodiac'] is 'Cancer':
        node_color.append('orangered')
    elif node[1]['zodiac'] is 'Leo':
        node_color.append('crimson')
    elif node[1]['zodiac'] is 'Virgo':
        node_color.append('mediumvioletred')
    elif node[1]['zodiac'] is 'Libra':
        node_color.append('darkviolet')
    elif node[1]['zodiac'] is 'Scorpio':
        node_color.append('mediumblue')
    elif node[1]['zodiac'] is 'Sagittarius':
        node_color.append('royalblue')
    elif node[1]['zodiac'] is 'Capricorn':
        node_color.append('c')
    elif node[1]['zodiac'] is 'Aquarius':
        node_color.append('green')
    elif node[1]['zodiac'] is 'Pisces':
        node_color.append('limegreen')
    else:
        #print('color for', node[1]['zodiac'], 'not found')
        node_color.append('grey')

In [ ]:
node_size = [v*0.5 for v in dict(G.degree).values()]

In [ ]:
forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=2.0,
                        strongGravityMode=False,
                        gravity=1.0,

                        # Log
                        verbose=False)
positions = forceatlas2.forceatlas2_networkx_layout(U, pos=None, iterations=200)

NameError: ignored

In [ ]:
nx.draw_networkx_nodes(U, pos=pos, alpha=0.3, node_size=node_size, node_color=node_color)
nx.draw_networkx_edges(U, pos=pos, alpha=0.7,edge_color=edge_color, width=0.2)

# Caption figure
plt.figtext(
    0.5, 
    0.1, 
    'Fig. todo: Visual representation of the celebrity network G using force Atlas 2 positions', 
    wrap=True, 
    horizontalalignment='center', 
    fontsize=12
)

#Create legend
color_legend = {
    'Aries': '#FFFF66', 'Taurus': 'gold', 'Gemini': 'orange', 'Cancer': 'orangered', 'Leo': 'crimson', 'Virgo': 'mediumvioletred', 'Libra': 'darkviolet', 'Scorpio': 'mediumblue', 'Sagittarius': 'royalblue', 'Capricorn': 'c', 'Aquarius': 'green', 'Pisces': 'limegreen'}
for label in color_legend:
    plt.plot([0],[0],color=color_legend[label],label=label)
plt.legend()

plt.show()

NameError: ignored

Compatability: Edge likelyhood depending on zodiac or element?
How to see if edge more likely between certain attributes?

# Word analysis

In [ ]:
### TWITTER TWEET TASKS

# Query tweets from twitter API
tweets = {}
for twitter_user in df.Handle:
    # Getting their 50 last tweets (todo: set a date to start retrieving, remove RT)
    tweets[twitter_user] = []
    for status in tweepy.Cursor(api.user_timeline, id = twitter_user, count=1).items(50):
        tweets[twitter_user].append(status.text)
        
# Add tweets to df
df['Tweets'] = df['Handle'].map(tweets)        

TweepError: ignored

# Clouds


In [ ]:
words = nltk.tokenize.word_tokenize(p)
fdist = FreqDist(words)
zodiac_signs = ['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo', 'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo']

for zodiac in zodiac_signs:
    mask_zodiac = np.array(Image.open("../" + zodiac + ".png"))

    # Create the wordclouds with a transparent background and the shape of the zodiac sign
    wc_zodiacs = WordCloud(mode="RGBA", background_color = None, max_words=200, mask=mask_zodiac, min_font_size=30, random_state=42,
            collocations = False).generate_from_frequencies(fdist)

    # Color the wordcloud with the colors of the zodiac sign
    image_colors = ImageColorGenerator(mask_zodiac)

    # Plot and save the wordcloud
    plt.figure(figsize = (17,17))
    plt.imshow(wc_zodiacs.recolor(color_func=image_colors), interpolation='bilinear')
    plt.axis("off")
    plt.show()
    plt.savefig("../" + zodiac + "_wordcloud.png")

In [ ]:
# SAVE / LOAD DF  (Better in Jupyter)
#df.to_pickle("../df.pkl")
uploaded_df = pd.read_pickle("../first_50_pages_df.pkl")

In [ ]:
#uploaded_df.head
#result = pd.concat([uploaded_df, df1])
#result.reset_index(drop=True, inplace=True)
#result.head

result.to_pickle("../100p_df.pkl")


In [ ]:
df = pd.read_csv("../100p_df.csv", error_bad_lines=False)
#df.drop_duplicates(subset ="WikiLink", inplace = True) 
print(df.head)


<bound method NDFrame.head of       Unnamed: 0                Name  ... Element           WikiLink
0              0        Barack Obama  ...    Fire       Barack Obama
1              1       Justin Bieber  ...   Water      Justin Bieber
2              2          KATY PERRY  ...   Water         Katy Perry
3              3             Rihanna  ...   Water            Rihanna
4              4   Cristiano Ronaldo  ...     Air  Cristiano Ronaldo
...          ...                 ...  ...     ...                ...
1484        1497  chaitanya akkineni  ...    Fire     Naga Chaitanya
1485        1498        bubba watson  ...   Water       Bubba Watson
1486        1499        Awal Ashaari  ...    Fire       Awal Ashaari
1487        1500             ASHANTI  ...     Air   Ashanti (singer)
1488        1501           Jay Bilas  ...   Earth          Jay Bilas

[1489 rows x 8 columns]>


In [ ]:
#df.drop(['Unnamed: 0'], axis=1, inplace=True)
#df.reset_index(drop=True, inplace=True)
print(df.head)
df.to_csv("../100p_df.csv")


<bound method NDFrame.head of                     Name          Handle  ...  Element           WikiLink
0           Barack Obama    @barackobama  ...     Fire       Barack Obama
1          Justin Bieber   @justinbieber  ...    Water      Justin Bieber
2             KATY PERRY      @katyperry  ...    Water         Katy Perry
3                Rihanna        @rihanna  ...    Water            Rihanna
4      Cristiano Ronaldo      @cristiano  ...      Air  Cristiano Ronaldo
...                  ...             ...  ...      ...                ...
1484  chaitanya akkineni  @chay_akkineni  ...     Fire     Naga Chaitanya
1485        bubba watson    @bubbawatson  ...    Water       Bubba Watson
1486        Awal Ashaari    @awalashaari  ...     Fire       Awal Ashaari
1487             ASHANTI        @ashanti  ...      Air   Ashanti (singer)
1488           Jay Bilas       @jaybilas  ...    Earth          Jay Bilas

[1489 rows x 7 columns]>
